# Optimal GD for barb

1. Determine the frequency $u$ for which the average FSC progressively computed from the normalized frequency 0.5 is higher than some given threshold $t=0.143$, by default.
2. Compute the corresponding Gaussian filter length $\tau$ whose cut-off frequency is $u$.
3. Filter the image.

In [ ]:
import time
from collections import namedtuple

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

import scipy.ndimage

try:
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker

try:
    from skimage import io as skimage_io
except:
    !pip install scikit-image
    from skimage import io as skimage_io

try:
    import information_theory as IT
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    import information_theory as IT

import utils        #freq, c_avg = fsc.get_SFRC_curve(denoised)
        #first_half = c_avg[:len(c_avg)>>1]
        #SFRC_curve.append(first_half)
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

In [ ]:
# apt install cm-super-minimal
# apt install dvipng
plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

In [ ]:
try:
    from self_fourier_shell_correlation import fsc_utils as fsc
except:
    !pip install "self_fourier_shell_correlation @ git+https://github.com/vicente-gonzalez-ruiz/self_fourier_shell_correlation"
    from self_fourier_shell_correlation import fsc_utils as fsc

In [ ]:
try:
    import denoising.image.gaussian as denoising
except:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
    import denoising.image.gaussian as denoising

In [ ]:
denoiser = denoising.Monochrome_Denoising(logger)

### barb

In [ ]:
#Args = namedtuple("args", ["input"])
#args = Args("barb_0MMPG.png")
Args = namedtuple("args", "input")
args = Args("http://www.hpca.ual.es/~vruiz/images/barb.png")

In [ ]:
#try:
#    Y = skimage_io.imread(args.input)
#except FileNotFoundError:
#    %run barb_0MMPG.ipynb
#    Y = skimage_io.imread(args.input)
X = skimage_io.imread(args.input)

### Compute SFRC

In [ ]:
freq, c_avg_Y = fsc.get_SFRC_curve(Y)

In [ ]:
plt.plot(freq, c_avg_Y)

In [ ]:
taus = [(0.2 + i/40) for i in range(40)] # Number of points per line
std_devs = [i for i in range(0, 100, 20)]
PCC_curves = []
SFRC_curves = []
poisson_ratio = 0.5
gamma = 0.15
for std_dev in std_devs: # Number of noise levels
    #gamma = std_dev / 50
    PCC_curve = []
    SFRC_curve = []
    Y = utils.generate_MPGN(X, std_dev, gamma, poisson_ratio).reshape(X.shape)
    #Y = np.clip(a = Y, a_min=0, a_max=255) # Probar a quitar
    #for sigma_kernel in range(5, 20, 1):
    for tau in taus: # Filter length
        #sigma_kernel /= 10
        sigma = np.array([tau, tau])
        kernel = [None]*2
        kernel[0] = get_gaussian_kernel(sigma[0])
        kernel[1] = get_gaussian_kernel(sigma[1])
        #print("Kernel:", kernel)
        denoised = denoiser.filter(Y, kernel)
        #PSNR = IT.distortion.PSNR(denoised, X)
        PCC = np.corrcoef(denoised.flatten(), X.flatten())[0, 1]
        print("std_dev:", std_dev, "sigma_kernel:", tau, "PCC:", PCC)
        PCC_curve.append(PCC)
        #freq, c_avg = fsc.get_SFRC_curve(denoised)
        #first_half = c_avg[:len(c_avg)>>1]
        #SFRC_curve.append(first_half)
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    PCC_curves.append(PCC_curve)
    #SFRC_curves.append(SFRC_curve)
    
    #sigma_index += 1
    print()

In [ ]:
#plt.title(r"$\mathbf{X}=\mathrm{Barb}$")
#plt.title(r"$\mathbb{E}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\mathrm{Barb})$")
#plt.title("Averaging Poisson noisy instances of \"Barb\"")
#for i in range(len(curves)):
i = 0
for std_dev in std_devs:
    #plt.plot([i/10 for i in range(5, 20, 1)], curves[i], label=rf"$\sigma={10+i*5}, \lambda={(10-i)/40}\cdot" + r"\mathrm{Barb}$")
    #if ((10+i*5) == 40) and ((10-i)/40 == 0.15):
    #plt.plot(sigmas_kernel, curves[i], label=rf"$\sigma={10+i*5}, \gamma={(10-i)/40}" + r", \mathrm{argmax}_\tau=" + rf"{sigmas_kernel[np.argmax(curves[i])]:.2f}$", marker='o')
    #else:
    plt.plot(taus,
        PCC_curves[i],
        label=rf"$\sigma={std_dev}, \gamma={gamma}"
        + r", \tau^*="
        + rf"{taus[np.argmax(PCC_curves[i])]:.2f}"
        + "$")
    i += 1
string  = r"$"
string += r"\mathrm{PCC}"
string += r"(\mathbf{X}, "
string += r"\mathrm{GD}_\tau"
string += r"(\hat{\mathbf{X}}"
#string += r"+ \mathbf{N}_{\mathcal{N}"
#string += r"(\sigma)}"
#string += r"+ \mathbf{N}_"
#string += r"{\mathcal{P}(\gamma\mathbf{X})}/\gamma))$"
string += r"))$"
plt.ylabel(string)
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("barb_GD_optimal.pdf", bbox_inches='tight')

In [ ]:
input()

In [ ]:
def gaussian_lowpass_filter(shape, normalized_cutoff):
    """Creates a Gaussian low-pass filter in the frequency domain using normalized frequency."""
    rows, cols = shape
    N = min(rows, cols)  # Assume square image size for cutoff scaling

    # Convert normalized frequency to pixel-based cutoff
    cutoff_px = (normalized_cutoff * N) / 2  

    # Create frequency coordinate grids
    u = np.arange(-cols//2, cols//2)
    v = np.arange(-rows//2, rows//2)
    U, V = np.meshgrid(u, v)

    # Compute Gaussian filter
    D = np.sqrt(U**2 + V**2)  # Distance from center
    H = np.exp(-(D**2) / (2 * (cutoff_px**2)))  # Gaussian function

    return H

In [ ]:
dft = np.fft.fft2(Y)
dft_shift = np.fft.fftshift(dft)

normalized_cutoff = 1.0  # Adjust this value as needed

# Generate Gaussian filter
H = gaussian_lowpass_filter(Y.shape, normalized_cutoff)
magnitude_spectrum = np.log(1 + np.abs(dft_shift))
plt.imshow(magnitude_spectrum)

In [ ]:
plt.imshow(H)

# Apply the filter
filtered_dft = dft_shift * H

# Inverse Fourier Transform
dft_inverse = np.fft.ifftshift(filtered_dft)
image_filtered = np.fft.ifft2(dft_inverse)
image_filtered = np.abs(image_filtered)

# Display results
plt.figure(figsize=(10,5))
plt.subplot(1,2,1), plt.imshow(Y, cmap='gray'), plt.title('Original Image')
plt.subplot(1,2,2), plt.imshow(image_filtered, cmap='gray'), plt.title(f'Filtered (Normalized Cutoff={normalized_cutoff})')
plt.show()

In [ ]:
def normalized_cutoff_to_tau(cutoff_freq, img_shape):
    N = min(img_shape)
    tau = 1 / (2 * np.pi * cutoff_freq * (N / 2))
    return tau

In [ ]:
tau = normalized_cutoff_to_tau(0.25, Y.shape)
tau

In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt

def get_gaussian_kernel(sigma=1):
    """Generates a 1D Gaussian kernel using scipy.ndimage.gaussian_filter1d()."""
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2:
        delta = np.zeros(number_of_coeffs)
        delta[delta.size // 2] = 1  # Impulse at center
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]  # Remove the first and last zero elements

# Define parameters
sigma_values = np.linspace(0.5, 10, 10)  # Range of sigma values to test
N = 256  # Signal length
normalized_cutoff_freqs = []

for sigma in sigma_values:
    # Generate Gaussian kernel
    kernel = get_gaussian_kernel(sigma)

    # Compute its frequency response
    freq_response = np.abs(np.fft.fft(kernel, N))  # Compute FFT
    freq_response = freq_response[:N//2]  # Take only the first half (positive frequencies)
    freq_axis = np.linspace(0, 0.5, N//2)  # Normalized frequency axis (Nyquist = 0.5)

    # Find the effective cut-off frequency (where magnitude falls below ~0.707)
    cutoff_idx = np.where(freq_response < freq_response[0] / np.sqrt(2))[0][0]
    cutoff_freq = freq_axis[cutoff_idx]
    normalized_cutoff_freqs.append(cutoff_freq)

# Fit an empirical formula: f_c^* ≈ C / sigma
coeff = np.polyfit(1 / sigma_values, normalized_cutoff_freqs, 1)

# Display empirical relationship
plt.figure(figsize=(6, 4))
plt.plot(sigma_values, normalized_cutoff_freqs, 'bo-', label='Measured Cut-off')
plt.plot(sigma_values, coeff[0] * (1 / sigma_values) + coeff[1], 'r--', label=f'Fit: {coeff[0]:.3f}/sigma + {coeff[1]:.3f}')
plt.xlabel("Sigma")
plt.ylabel(r"Effective Normalized Cut-off Frequency ($f_c^*$)")
plt.legend()
plt.title("Empirical Relationship Between Sigma and Cut-off Frequency")
plt.grid()
plt.show()

print(f"Empirical formula: f_c^* ≈ {coeff[0]:.3f} / sigma + {coeff[1]:.3f}")


In [ ]:
def normalized_cutoff_to_tau(cutoff_freq):
    tau = 0.158 / cutoff_freq
    return tau

In [ ]:
tau = normalized_cutoff_to_tau(0.25)
tau

In [ ]:
tau = 0.93

In [ ]:
sigma = np.array([tau, tau])
kernel = [None]*2
kernel[0] = utils.get_gaussian_kernel(sigma[0])
kernel[1] = utils.get_gaussian_kernel(sigma[1])
print(np.sum(kernel[0]))
plt.plot(kernel[0])
plt.show()

In [ ]:
denoised = denoiser.filter(Y, kernel)

In [ ]:
freq, c_avg_denoised = fsc.get_SFRC_curve(denoised)

In [ ]:
plt.plot(freq, c_avg_Y)
plt.plot(freq, c_avg_denoised)

In [ ]:
PSNR1 = IT.distortion.PSNR(denoised1, X1)

In [ ]:
string  = r"$\mathrm{GD}_{\text{"
string += str(TAU)
string += r"}}(\hat{\mathbf{X}}_1)"
#string += r"}}(\mathrm{Barb} + \mathbf{N}_{\mathcal{N}"
#string += r"(\sigma"
#string += f"={std_dev})"
#string += r'}'
#string += r" + \mathbf{N}_{\mathcal{P}"
#string += r"(\lambda"
#string += f"={gamma}"
#string += r'\cdot\mathrm{Barb})}'
#string += rf"$_{(\sigma^2={std_dev}^2)}$"
#string += f"/{gamma})"
string += f",~{IT.distortion.PSNR(Y1.astype(np.uint8), X1):.2f}"
string += r"~\text{dB}$"
plt.title(f"{string}")
plt.imshow(denoised1, cmap="gray")
plt.savefig('GD_0MMPG_barb.pdf', bbox_inches='tight')

In [ ]:
from scipy import signal

In [ ]:
w1, h1 = signal.freqz(kernel[0], fs=10000)

In [ ]:
plt.plot(w1, 20 * np.log10(abs(h1)), 'b')

In [ ]:
denoised2 = denoiser.filter(Y2, kernel)

In [ ]:
PSNR2 = IT.distortion.PSNR(denoised2, X2)

In [ ]:
string  = r"$\mathrm{GD}_{\text{"
string += str(TAU)
string += r"}}(\hat{\mathbf{X}}_2)"
#string += r"}}(\mathrm{Barb} + \mathbf{N}_{\mathcal{N}"
#string += r"(\sigma"
#string += f"={std_dev})"
#string += r'}'
#string += r" + \mathbf{N}_{\mathcal{P}"
#string += r"(\lambda"
#string += f"={gamma}"
#string += r'\cdot\mathrm{Barb})}'
#string += rf"$_{(\sigma^2={std_dev}^2)}$"
#string += f"/{gamma})"
string += f",~{IT.distortion.PSNR(Y2.astype(np.uint8), X2):.2f}"
string += r"~\text{dB}$"
plt.title(f"{string}")
plt.imshow(denoised2, cmap="gray")
plt.savefig('GD_0MMPG_lake.pdf', bbox_inches='tight')

#### GD_PCC_0MMPG_barb and GD_PCC_0MMPG_lake

In [ ]:
sigmas_kernel = [(0.2 + i/40) for i in range(40)] # Number of points per line
#sigmas_kernel.append(0.625)
#sigmas_kernel.sort()
#sigmas_kernel = [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
std_devs = [i for i in range(0, 100, 20)]
PCC_curves1 = []
PCC_curves2 = []
#SFRC_curves = []
poisson_ratio = 0.5
gamma = 0.15
for std_dev in std_devs: # Number of noise levels
    #gamma = std_dev / 50
    PCC_curve1 = []
    PCC_curve2 = []
    #SFRC_curve = []
    Y1 = utils.generate_MPGN(X1, std_dev, gamma, poisson_ratio).reshape(X1.shape)
    Y2 = utils.generate_MPGN(X2, std_dev, gamma, poisson_ratio).reshape(X2.shape)
    #Y = np.clip(a = Y, a_min=0, a_max=255) # Probar a quitar
    #for sigma_kernel in range(5, 20, 1):
    for sigma_kernel in sigmas_kernel: # Filter length
        #sigma_kernel /= 10
        sigma = np.array([sigma_kernel, sigma_kernel])
        kernel = [None]*2
        kernel[0] = utils.get_gaussian_kernel(sigma[0])
        kernel[1] = utils.get_gaussian_kernel(sigma[1])
        #print("Kernel:", kernel)
        denoised1 = denoiser.filter(Y1, kernel)
        denoised2 = denoiser.filter(Y2, kernel)
        #PSNR = IT.distortion.PSNR(denoised, X)
        PCC1 = np.corrcoef(denoised1.flatten(), X1.flatten())[0, 1]
        PCC2 = np.corrcoef(denoised2.flatten(), X2.flatten())[0, 1]
        print("std_dev:", std_dev, "sigma_kernel:", sigma_kernel, "PCC1:", PCC1, "PCC2:", PCC2)
        PCC_curve1.append(PCC1)
        PCC_curve2.append(PCC2)
        #freq, c_avg = fsc.get_SFRC_curve(denoised)
        #first_half = c_avg[:len(c_avg)>>1]
        #SFRC_curve.append(first_half)
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    PCC_curves1.append(PCC_curve1)
    PCC_curves2.append(PCC_curve2)
    #SFRC_curves.append(SFRC_curve)
    
    #sigma_index += 1
    print()

In [ ]:
#plt.title(r"$\mathbf{X}=\mathrm{Barb}$")
#plt.title(r"$\mathbb{E}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\mathrm{Barb})$")
#plt.title("Averaging Poisson noisy instances of \"Barb\"")
#for i in range(len(curves)):
i = 0
for std_dev in std_devs:
    #plt.plot([i/10 for i in range(5, 20, 1)], curves[i], label=rf"$\sigma={10+i*5}, \lambda={(10-i)/40}\cdot" + r"\mathrm{Barb}$")
    #if ((10+i*5) == 40) and ((10-i)/40 == 0.15):
    #plt.plot(sigmas_kernel, curves[i], label=rf"$\sigma={10+i*5}, \gamma={(10-i)/40}" + r", \mathrm{argmax}_\tau=" + rf"{sigmas_kernel[np.argmax(curves[i])]:.2f}$", marker='o')
    #else:
    plt.plot(sigmas_kernel,
        PCC_curves1[i],
        label=rf"$\sigma={std_dev}, \gamma={gamma}"
        + r", \tau^*="
        + rf"{sigmas_kernel[np.argmax(PCC_curves1[i])]:.2f}"
        + "$")
    i += 1
string  = r"$"
string += r"\mathrm{PCC}"
string += r"(\mathbf{X}_1, "
string += r"\mathrm{GD}_\tau"
string += r"(\mathbf{X}_1"
string += r"+ \mathbf{N}_{\mathcal{N}"
string += r"(\sigma)}"
string += r"+ \mathbf{N}_"
string += r"{\mathcal{P}(\gamma\mathbf{X}_1)}/\gamma))"
string += r"$"
plt.ylabel(string)
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("GD_PCC_0MMPG_barb.pdf", bbox_inches='tight')

In [ ]:
#plt.title(r"$\mathbf{X}=\mathrm{Barb}$")
#plt.title(r"$\mathbb{E}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\mathrm{Barb})$")
#plt.title("Averaging Poisson noisy instances of \"Barb\"")
#for i in range(len(curves)):
i = 0
for std_dev in std_devs:
    #plt.plot([i/10 for i in range(5, 20, 1)], curves[i], label=rf"$\sigma={10+i*5}, \lambda={(10-i)/40}\cdot" + r"\mathrm{Barb}$")
    #if ((10+i*5) == 40) and ((10-i)/40 == 0.15):
    #plt.plot(sigmas_kernel, curves[i], label=rf"$\sigma={10+i*5}, \gamma={(10-i)/40}" + r", \mathrm{argmax}_\tau=" + rf"{sigmas_kernel[np.argmax(curves[i])]:.2f}$", marker='o')
    #else:
    plt.plot(sigmas_kernel,
        PCC_curves2[i],
        label=rf"$\sigma={std_dev}, \gamma={gamma}"
        + r", \tau^*="
        + rf"{sigmas_kernel[np.argmax(PCC_curves2[i])]:.2f}"
        + "$")
    i += 1
string  = r"$"
string += r"\mathrm{PCC}"
string += r"(\mathbf{X}_2, "
string += r"\mathrm{GD}_\tau"
string += r"(\mathbf{X}_2"
string += r"+ \mathbf{N}_{\mathcal{N}"
string += r"(\sigma)}"
string += r"+ \mathbf{N}_"
string += r"{\mathcal{P}(\gamma\mathbf{X}_2)}/\gamma))"
string += r"$"
plt.ylabel(string)
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("GD_PCC_0MMPG_lake.pdf", bbox_inches='tight')